# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = "../WeatherPy/Output/cities.csv"
weather_data = pd.read_csv(weather_file)
weather_data.dropna(inplace = True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Lat","Lng"]].astype(float)
Humidity = weather_data["Humidity"].astype(float)

In [14]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, 
                                 weights=Humidity, 
                                 dissipating=False, 
                                 max_intensity=100,
                                 point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
vacation_cities = weather_data.loc[(weather_data["Max Temp"] <= 78 ) 
                                   & (weather_data["Max Temp"] >= 68)
                                  & (weather_data["Wind Speed"] <= 10)
                                  & (weather_data["Cloudiness"] == 0)]     
vacation_cities

,City,Lat,Lng,Temperature,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,port alfred,-42.7826,147.0587,73.45,75.99,33.0,0.0,3.00,AU,2021-02-14 02:36:37
91,sinnamary,-18.4750,-70.3042,71.60,71.60,60.0,0.0,9.22,CL,2021-02-14 02:37:27
151,mao,-24.8716,25.8699,69.80,69.80,78.0,0.0,6.91,BW,2021-02-14 02:38:41
160,mys shmidta,15.7432,-96.4661,70.63,70.63,82.0,0.0,6.53,MX,2021-02-14 02:38:53
165,sola,-32.8833,-68.8000,73.40,73.40,60.0,0.0,2.30,AR,2021-02-14 02:38:59
171,whitehorse,12.0708,1.7889,74.55,74.55,17.0,0.0,6.42,BF,2021-02-14 02:39:06
180,krasnoselkup,-33.9180,25.5701,73.40,73.40,94.0,0.0,4.61,ZA,2021-02-14 02:39:17
221,vanavara,20.1167,77.7167,74.03,74.03,29.0,0.0,6.93,IN,2021-02-14 02:40:10
226,muros,5.5167,5.7500,77.95,77.95,89.0,0.0,0.69,NG,2021-02-14 02:40:17
260,severo-kurilsk,14.8961,-24.4956,69.96,69.96,75.0,0.0,9.51,CV,2021-02-14 02:36:05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_name = []

In [20]:
# params dictionary to update each iteration
vaca_params = {"radius": 50000,"types": "lodging","key": g_key}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_cities.iterrows():
    
    # get lat, lng from df - this is what isn't working
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    vaca_params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params = vaca_params)

In [21]:
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_name.append(name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

SyntaxError: invalid syntax (<ipython-input-21-51e52d587ff9>, line 4)

In [22]:
hotel_df = {"Hotel": hotel_name}

In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'dict' object has no attribute 'iterrows'

In [24]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,info_box_contents=[f"Hotel: {hotel_name}" for 
                                                                 hotel in hotel_df"])
fig.add_layer(markers)
# Display figure
fig

SyntaxError: EOL while scanning string literal (<ipython-input-24-2099cd0d449a>, line 4)